In [1]:
!git clone "https://github.com/dhoard123/FinalProject.git"


Cloning into 'FinalProject'...


IMPORTS

In [ ]:
from google.colab import drive
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import cv2
import scipy
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.dataset import Dataset
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import pandas as pd
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

PRE-PROCESS

In [ ]:
def get_data(csv_file):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []

    data_info = pd.read_csv(csv_file, skiprows=1, header=None)
    image_arr = np.asarray(data_info.iloc[:, 0])
    label_arr = np.asarray(data_info.iloc[:, 7])
    height = 200
    width = 200

    for index, image_name in enumerate(image_arr):
        #print(index, str(BASE_DIR / 'images' / 'train' / str(image_name + '.jpg')))
        img_file = cv2.imread(str(BASE_DIR / 'images' / 'train' / str(image_name + '.jpg')))
        label = label_arr[index]
        print(index, image_name, label)

        if img_file is not None:
            img_file = resize(img_file, (height, width, 3), order=1)
            img_np_arr = np.asarray(img_file)
            X.append(img_np_arr)
            y.append(int(label))
    
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

TRAIN OR EVALUATE MODEL

In [ ]:
train_eval = str(input("Training mode (0) or eval mode [must have .npy files] (1)?: "))
while train_eval != '0' and train_eval != '1':
    train_eval = str(input("Training mode (0) or eval mode [must have .npy files] (1)?: "))
train_eval = int(train_eval)

Training mode (0) or eval mode [must have .npy files] (1)?: 0


HYPER PARAMETERS

In [ ]:
# Hyper parameters
num_epochs = 5
num_classes = 2
batch_size = 25
learning_rate = 0.001

In [ ]:
print(BASE_DIR)

/content/gdrive/My Drive/CS184A


TRAIN OR LOAD

In [ ]:
if train_eval == 0:
    
    # Ask whether to run the get_data or load from existing files
    save_load = str(input("[TAKES A LONG TIME] Create arrays and save (0) or load arrays from .npy files (1)?: "))
    while save_load != '0' and save_load != '1':
        save_load = str(input("[TAKES A LONG TIME] Create arrays and save (0) or load arrays from .npy files (1)?: "))
    save_load = int(save_load)

else:
    save_load = 1

[TAKES A LONG TIME] Create arrays and save (0) or load arrays from .npy files (1)?: 1


READ AND SAVE LABELED DATA

In [ ]:
if save_load == 0:
    X_train, y_train = get_data(BASE_DIR / 'train.csv')
    np.save(str(BASE_DIR / 'X_train.npy'), X_train)
    np.save(str(BASE_DIR / 'y_train.npy'), y_train)

    X_test, y_test = get_data(BASE_DIR / 'test.csv')
    np.save(str(BASE_DIR / 'X_test.npy'), X_test)
    np.save(str(BASE_DIR / 'y_test.npy'), y_test)

    print("Preliminary data saved to", str(BASE_DIR))
elif save_load == 1:
    X_train = np.load(str(BASE_DIR / 'X_train.npy'))
    y_train = np.load(str(BASE_DIR / 'y_train.npy'))
    X_test = np.load(str(BASE_DIR / 'X_test.npy'))
    y_test = np.load(str(BASE_DIR / 'y_test.npy'))
    print("Preliminary data loaded from", str(BASE_DIR))

FileNotFoundError: ignored

In [ ]:
print()
print("Training shape:", y_train.shape, X_train.shape)
print("Test shape:",y_test.shape, X_test.shape)

In [ ]:
encoder = LabelEncoder()  # convert labels to 0 to n-1
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

TRAIN AND TEST DATALOADER

In [ ]:
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train.transpose(0,3,1,2)).float(), torch.from_numpy(y_train))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_test.transpose(0,3,1,2)).float(), torch.from_numpy(y_test))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,                                           
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Convolutional Neural Network

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=5, stride=1, padding=2), # 3 x 200 x 200
            #nn.BatchNorm2d(8),
            #nn.ReLU(),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 8 x 100 x 100
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=5, stride=1, padding=2), # 16 x 100 x 100
            #nn.BatchNorm2d(32),
            #nn.ReLU(),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 16 x 50 x 50
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), # 32 x 50 x 50
            #nn.BatchNorm2d(32),
            #nn.ReLU(),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 32 x 25 x 25
        self.fc = nn.Linear(25*25*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
test_step = len(test_loader)

TRAINING MODE

In [ ]:
if train_eval == 0:
    # Train the model
    
    for epoch in range(num_epochs):   
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


    # AutoSave model
    torch.save(model.state_dict(), BASE_DIR / "model_3.pt")

    
    # Test the model
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the {} test images: {} %'.format((test_step * 25), (100 * correct / total)))


    # Print model's state_dict
    print("Model's state_dict:")
    for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())


EVALUATION MODE

In [ ]:
elif train_eval == 1:
    model = ConvNet(num_classes).to(device)
    model.load_state_dict(torch.load(BASE_DIR / "model_3.pt"))

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the ~{} test images: {} %'.format((test_step * 25), (100 * correct / total)))
